In [ ]:
#You can !pip install these packages if you do not have them. Or if in Jupyter Notebook, go to Environments in Anaconda Navigator
import netCDF4 as nc
import numpy as np
import os
os.environ["PROJ_LIB"] = "C:\\Utilities\\Python\\Anaconda\\Library\\share"; #fixr
import plotly.express as px
from numpy import linspace
from numpy import meshgrid
from netCDF4 import Dataset as ds

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from mpl_toolkits.basemap import Basemap

## Setting up meshgrid, image projections, and colorscale before graphing

In [ ]:
#loading in Air meshgrids
AirFile = nc.Dataset('data/air.mon.meanv3.nc')
air_lats = AirFile.variables['lat'][:]
air_lons = AirFile.variables['lon'][:]
pressure = AirFile.variables['level'][:]
airsize = AirFile.variables['air'][0].size
xxair, yyair = np.meshgrid(air_lons, air_lats)
#loading in Water meshgrids
WaterFile = nc.Dataset('data/pottmp.1998.nc') #arbitary year chosen. All years have same dimensions
water_lats = WaterFile.variables['lat'][:]
water_lons = WaterFile.variables['lon'][:]
depths = WaterFile.variables['level'][:]
xxwater, yywater = np.meshgrid(water_lons, water_lats)

In [ ]:
#obtaining image background set from basemap
import skimage.io as sio
image = sio.imread('data/Basemap Background.PNG')
image.shape
x = np.linspace(0,360, image.shape[1])
y = np.linspace(-90,90, image.shape[0])
xximage, yyimage = np.meshgrid(x,y)

In [ ]:
#Number of colors, choice color, etc can be adjusted. 
colors = [[0, "black"],[0.01, "violet"],[1/6+0.05, "blue"],[1/3, "cyan"],
          [1/2, "white"],[2/3, "yellow"],[5/6, "orange"],[1, "red"]]

## Loading in and altering files for graphing

In [ ]:
#This is just for January. Also works for other months if have files
EOFs = np.load('data/January Physical EOFs.npy')

In [ ]:
#For this example, we are graphing January EOF Mode 1. 
#For different modes, change second index to (mode # -1) 
lim = np.minimum(np.abs(np.nanmin(EOFs[:,0])), np.nanmax(EOFs[:,0]))
PhysicalEOFs = np.clip(EOFs[:,0], -lim, lim)

In [ ]:
#Done to make values extreme as Plotly reads NaNs as 0's when graphing.
for j in range(0, PhysicalEOFs.shape[0]):
    if np.isnan(PhysicalEOFs[j,0]):
        PhysicalEOFs[j,0] = 1 #set to positive 1 because EOF1 is flipped. 
        # When doing EOF2, set to negative 1. Check each EOF.

## Plotting Air-Ocean Coupled EOF1 (Figure 2)

In [ ]:
fig = go.Figure(data=[
    go.Surface(x = xxwater, y= yywater, z=-5*np.ones(xxwater.shape),
               surfacecolor =0-np.reshape(PhysicalEOFs[airsize: airsize+418*360], (418,360)) ,
               colorscale = colors,  opacity = 1),
])
fig.add_trace(go.Surface(x = xximage, y = yyimage, z = 124.5 *np.ones(xximage.shape), 
                        surfacecolor = np.flipud(image[:,:,0])*lim/255, 
                        colorscale = 'Greys_r', showscale = False, opacity = 1))
j = 0
while (pressure[j] >= 10):
    layer = j*5*np.ones(xxair.shape)
    fig.add_trace(go.Surface(x =xxair, y =yyair, z=layer,
                             surfacecolor =0-np.reshape(PhysicalEOFs[j*360*181: (j+1)*360*181],(181,360)),
                             colorscale = colors, showscale = False, opacity = .5))
    j+=1
fig.update_traces(cmax= lim, selector=dict(type='surface'))
fig.update_traces(cmin= -lim, selector=dict(type='surface'))
fig.update_traces(colorbar_exponentformat="power", colorbar_showexponent= "none")
fig.update_traces(colorbar_title_text=r"x10⁻⁵", colorbar_title_font = dict(size =25))
fig.update_traces(colorbar_tickfont = dict(size = 25))
fig.update_layout(scene = dict(aspectratio = dict(x = 2, y = 1, z = 1.5)))

fig.update_layout(title = 'January EOF1')
fig.update_layout(scene = dict(
                xaxis_title='Longitude',
                yaxis_title='Latitude',
                zaxis_title='Pressure'))
fig.update_layout(template='plotly_dark')
# Different types of customized ticks
fig.update_layout(scene = dict(
                    zaxis = dict(
                        ticktext= ['1000','850','500','250', '50', '10'],
                        tickvals= [0,25,60, 85, 110,125])))
fig.update_scenes(xaxis = dict(tickfont=dict(size=14),titlefont=dict(size=16)))
fig.update_scenes(yaxis = dict(tickfont=dict(size=14),titlefont=dict(size=16)))
fig.update_scenes(zaxis = dict(tickfont=dict(size=14),titlefont=dict(size=16)))
fig.show()
fig.write_html("Jan 3D EOF1.html") 

## Plotting Air-Ocean Coupled EOF2 (Figure 7)

In [ ]:
lim = np.minimum(np.abs(np.nanmin(EOFs[:,1])), np.nanmax(EOFs[:,1]))
PhysicalEOFs = np.clip(EOFs[:,1], -lim, lim)

In [ ]:
#Done to make values extreme as Plotly reads NaNs as 0's when graphing.
for j in range(0, PhysicalEOFs.shape[0]):
    if np.isnan(PhysicalEOFs[j,0]):
        PhysicalEOFs[j,0] = -1 

In [ ]:
fig = go.Figure(data=[
    go.Surface(x = xxwater, y= yywater, z=-5*np.ones(xxwater.shape),
               surfacecolor =np.reshape(PhysicalEOFs[airsize: airsize+418*360], (418,360)) ,
               colorscale = colors,  opacity = 1),
])
fig.add_trace(go.Surface(x = xximage, y = yyimage, z = 124.5 *np.ones(xximage.shape), 
                        surfacecolor = np.flipud(image[:,:,0])*lim/255, 
                        colorscale = 'Greys_r', showscale = False, opacity = 1))
j = 0
while (pressure[j] >= 10):
    layer = j*5*np.ones(xxair.shape)
    fig.add_trace(go.Surface(x =xxair, y =yyair, z=layer,
                             surfacecolor =np.reshape(PhysicalEOFs[j*360*181: (j+1)*360*181],(181,360)),
                             colorscale = colors, showscale = False, opacity = .5))
    j+=1
fig.update_traces(cmax= lim, selector=dict(type='surface'))
fig.update_traces(cmin= -lim, selector=dict(type='surface'))
fig.update_traces(colorbar_exponentformat="power", colorbar_showexponent= "none")
fig.update_traces(colorbar_title_text=r"x10⁻⁵", colorbar_title_font = dict(size =25))
fig.update_traces(colorbar_tickfont = dict(size = 25))
fig.update_layout(scene = dict(aspectratio = dict(x = 2, y = 1, z = 1.5)))

fig.update_layout(title = 'January EOF2')
fig.update_layout(scene = dict(
                xaxis_title='Longitude',
                yaxis_title='Latitude',
                zaxis_title='Pressure'))
fig.update_layout(template='plotly_dark')
# Different types of customized ticks
fig.update_layout(scene = dict(
                    zaxis = dict(
                        ticktext= ['1000','850','500','250', '50', '10'],
                        tickvals= [0,25,60, 85, 110,125])))
fig.update_scenes(xaxis = dict(tickfont=dict(size=14),titlefont=dict(size=16)))
fig.update_scenes(yaxis = dict(tickfont=dict(size=14),titlefont=dict(size=16)))
fig.update_scenes(zaxis = dict(tickfont=dict(size=14),titlefont=dict(size=16)))
fig.show()
fig.write_html("Jan 3D EOF2.html") 

## Plotting Air Only EOF1 (Figure 10)

In [ ]:
#Loading in EOFs that do not have ocean coupling
EOFs = np.load('data/January Air Only Physical EOFs.npy')
#limitting to EOF1
lim = np.minimum(np.nanmax(EOFs[:,0]), np.abs(np.nanmin(EOFs[:,0])))
PhysicalEOFs = np.clip(EOFs[:,0], -lim, lim)

#No NaNs to worry about because no water data.

In [ ]:
fig1 = go.Figure(data=[
    go.Surface(x = xxair, y= yyair, z=np.zeros(xxair.shape),
               surfacecolor =0-np.reshape(PhysicalEOFs[0: 181*360], (181,360)) ,
               colorscale = colors,  opacity = 1),
])
fig1.add_trace(go.Surface(x = xximage, y = yyimage, z = 124.5 *np.ones(xximage.shape), 
                        surfacecolor = np.flipud(image[:,:,0])*lim/255, 
                        colorscale = 'Greys_r', showscale = False, opacity = 1))
j = 1
while (pressure[j] >= 10):
    layer = j*5*np.ones(xxair.shape)
    fig1.add_trace(go.Surface(x =xxair, y =yyair, z=layer,
                             surfacecolor =0-np.reshape(PhysicalEOFs[j*360*181: (j+1)*360*181],(181,360)),
                             colorscale = colors, showscale = False, opacity = .5))
    j+=1
fig1.update_traces(cmax= lim, selector=dict(type='surface'))
fig1.update_traces(cmin= -lim, selector=dict(type='surface'))
fig1.update_traces(colorbar_exponentformat="power", colorbar_showexponent= "none")
fig1.update_traces(colorbar_title_text=r"x10⁻⁴", colorbar_title_font = dict(size =25))
fig1.update_traces(colorbar_tickfont = dict(size = 25))
fig1.update_layout(scene = dict(aspectratio = dict(x = 2, y = 1, z = 1.5)))
fig1.update_layout(template='plotly_dark')
fig1.update_layout(title = 'January EOF1')
fig1.update_layout(scene = dict(
                xaxis_title='Longitude',
                yaxis_title='Latitude',
                zaxis_title='Pressure (mbar)'))

# Different types of customized ticks
fig1.update_layout(scene = dict(
                    zaxis = dict(
                        ticktext= ['1000','850','500','250', '50', '10'],
                        tickvals= [0,25,60, 85, 110,125])))
fig1.update_scenes(xaxis = dict(tickfont=dict(size=14),titlefont=dict(size=16)))
fig1.update_scenes(yaxis = dict(tickfont=dict(size=14),titlefont=dict(size=16)))
fig1.update_scenes(zaxis = dict(tickfont=dict(size=14),titlefont=dict(size=16)))
fig1.write_html("Jan Air Only EOF1.html")

## Plotting Air Only EOF2 (Figure 15)

In [ ]:
lim = np.minimum(np.nanmax(EOFs[:,1]), np.abs(np.nanmin(EOFs[:,1])))
PhysicalEOFs = np.clip(EOFs[:,1], -lim, lim)

In [ ]:
fig1 = go.Figure(data=[
    go.Surface(x = xxair, y= yyair, z=np.zeros(xxair.shape),
               surfacecolor =np.reshape(PhysicalEOFs[0: 181*360], (181,360)) ,
               colorscale = colors,  opacity = 1),
])
fig1.add_trace(go.Surface(x = xximage, y = yyimage, z = 124.5 *np.ones(xximage.shape), 
                        surfacecolor = np.flipud(image[:,:,0])*lim/255, 
                        colorscale = 'Greys_r', showscale = False, opacity = 1))
j = 1
while (pressure[j] >= 10):
    layer = j*5*np.ones(xxair.shape)
    fig1.add_trace(go.Surface(x =xxair, y =yyair, z=layer,
                             surfacecolor =np.reshape(PhysicalEOFs[j*360*181: (j+1)*360*181],(181,360)),
                             colorscale = colors, showscale = False, opacity = .5))
    j+=1
fig1.update_traces(cmax= lim, selector=dict(type='surface'))
fig1.update_traces(cmin= -lim, selector=dict(type='surface'))
fig1.update_traces(colorbar_exponentformat="power", colorbar_showexponent= "none")
fig1.update_traces(colorbar_title_text=r"x10⁻⁴", colorbar_title_font = dict(size =25))
fig1.update_traces(colorbar_tickfont = dict(size = 25))
fig1.update_layout(scene = dict(aspectratio = dict(x = 2, y = 1, z = 1.5)))
fig1.update_layout(template='plotly_dark')
fig1.update_layout(title = 'January EOF1')
fig1.update_layout(scene = dict(
                xaxis_title='Longitude',
                yaxis_title='Latitude',
                zaxis_title='Pressure (mbar)'))

# Different types of customized ticks
fig1.update_layout(scene = dict(
                    zaxis = dict(
                        ticktext= ['1000','850','500','250', '50', '10'],
                        tickvals= [0,25,60, 85, 110,125])))
fig1.update_scenes(xaxis = dict(tickfont=dict(size=14),titlefont=dict(size=16)))
fig1.update_scenes(yaxis = dict(tickfont=dict(size=14),titlefont=dict(size=16)))
fig1.update_scenes(zaxis = dict(tickfont=dict(size=14),titlefont=dict(size=16)))
fig1.write_html("Jan Air Only EOF1.html")

## Climatology

In [ ]:
clim = np.load('data/January Climatology.npy')

In [ ]:
fig = go.Figure(data=[
    go.Surface(x = xxwater, y= yywater, z=-5*np.ones(xxwater.shape),
               surfacecolor =np.reshape(clim[airsize: airsize+418*360], (418,360)) ,
               colorscale = colors,  opacity = 1),
])
fig.add_trace(go.Surface(x = xximage, y = yyimage, z = 124.5 *np.ones(xximage.shape), 
                        surfacecolor = np.flipud(image[:,:,0]), 
                        colorscale = 'Greys_r', showscale = False, opacity = 1))
j = 0
while (pressure[j] >= 10):
    layer = j*5*np.ones(xxair.shape)
    fig.add_trace(go.Surface(x =xxair, y =yyair, z=layer,
                             surfacecolor =np.reshape(clim[j*360*181: (j+1)*360*181],(181,360)),
                             colorscale = colors, showscale = False, opacity = .5))
    j+=1
fig.update_traces(cmax= 310, selector=dict(type='surface')) #top avg temp
fig.update_traces(cmin= 188, selector=dict(type='surface')) #bottom avg temp
fig.update_traces(colorbar_exponentformat="power", colorbar_showexponent= "none")
fig.update_traces(colorbar_title_text=r"K", colorbar_title_font = dict(size =25))
fig.update_traces(colorbar_tickfont = dict(size = 25))
fig.update_layout(scene = dict(aspectratio = dict(x = 2, y = 1, z = 1.5)))

fig.update_layout(title = 'January Climatology')
fig.update_layout(scene = dict(
                xaxis_title='Longitude',
                yaxis_title='Latitude',
                zaxis_title='Pressure'))
fig.update_layout(template='plotly_dark')
# Different types of customized ticks
fig.update_layout(scene = dict(
                    zaxis = dict(
                        ticktext= ['1000','850','500','250', '50', '10'],
                        tickvals= [0,25,60, 85, 110,125])))
fig.update_scenes(xaxis = dict(tickfont=dict(size=14),titlefont=dict(size=16)))
fig.update_scenes(yaxis = dict(tickfont=dict(size=14),titlefont=dict(size=16)))
fig.update_scenes(zaxis = dict(tickfont=dict(size=14),titlefont=dict(size=16)))
fig.show()
fig.write_html("January Climatology.html") 

## Standard Deviation

In [ ]:
st_dev = np.load('data/January St Dev.npy')

In [ ]:
fig = go.Figure(data=[
    go.Surface(x = xxwater, y= yywater, z=-5*np.ones(xxwater.shape),
               surfacecolor =np.reshape(clim[airsize: airsize+418*360], (418,360)) ,
               colorscale = colors,  opacity = 1),
])
fig.add_trace(go.Surface(x = xximage, y = yyimage, z = 124.5 *np.ones(xximage.shape), 
                        surfacecolor = np.flipud(image[:,:,0])/45, 
                        colorscale = 'Greys_r', showscale = False, opacity = 1))
j = 0
while (pressure[j] >= 10):
    layer = j*5*np.ones(xxair.shape)
    fig.add_trace(go.Surface(x =xxair, y =yyair, z=layer,
                             surfacecolor =np.reshape(clim[j*360*181: (j+1)*360*181],(181,360)),
                             colorscale = colors, showscale = False, opacity = .5))
    j+=1
fig.update_traces(cmax= 6, selector=dict(type='surface')) #top avg temp
fig.update_traces(cmin= 0, selector=dict(type='surface')) #bottom avg temp
fig.update_traces(colorbar_exponentformat="power", colorbar_showexponent= "none")
fig.update_traces(colorbar_title_text=r"K", colorbar_title_font = dict(size =25))
fig.update_traces(colorbar_tickfont = dict(size = 25))
fig.update_layout(scene = dict(aspectratio = dict(x = 2, y = 1, z = 1.5)))

fig.update_layout(title = 'January Standard Deviation')
fig.update_layout(scene = dict(
                xaxis_title='Longitude',
                yaxis_title='Latitude',
                zaxis_title='Pressure'))
fig.update_layout(template='plotly_dark')
# Different types of customized ticks
fig.update_layout(scene = dict(
                    zaxis = dict(
                        ticktext= ['1000','850','500','250', '50', '10'],
                        tickvals= [0,25,60, 85, 110,125])))
fig.update_scenes(xaxis = dict(tickfont=dict(size=14),titlefont=dict(size=16)))
fig.update_scenes(yaxis = dict(tickfont=dict(size=14),titlefont=dict(size=16)))
fig.update_scenes(zaxis = dict(tickfont=dict(size=14),titlefont=dict(size=16)))
fig.show()
fig.write_html("January Standard Deviation.html") 